In [41]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import torch
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn, optim
import torch.nn.functional as F

import torchvision
from torchvision import transforms

from sklearn.datasets import fetch_openml

In [38]:
X, y = fetch_openml(data_id=40996, return_X_y=True, as_frame=False) 

In [43]:
df = pd.DataFrame(X)
df['y'] = y
df = df.loc[df.index[df['y']=='4'].union(df.index[df['y']=='7'])]
df = df.reset_index()
df

,index,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,y
0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
1,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
2,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,56.0,67.0,22.0,0.0,2.0,0.0,0.0,0.0,0.0,4
3,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,149.0,159.0,0.0,0.0,0.0,0.0,0.0,4
4,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,163.0,165.0,53.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,69966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4
13996,69968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
13997,69974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
13998,69986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [ ]:
X_4_7 = torch.tensor(minmax_scale(df.drop("y", axis=1)).astype(np.float32)).reshape(B.shape[0],1,28,28)
y_4_7 = torch.tensor(B.y.replace(['5','9'],[0,1]).values.astype(np.float32)).unsqueeze(-1)

In [28]:
def trn_crval_tst(net, epochs, criterion, optimizer, trn_iter, crv_iter, tst_iter):
    trn_loss, crv_loss, tst_loss = [], [], []

    for e in range(1, epochs+1):
        print(" Train: ", e, '/', epochs)
        _, losses = train_test(net, 1, criterion, optimizer, trn_iter)
        trn_loss += [ losses[-1] ]
        print(" CrssVal: ", e, '/', epochs)
        _, losses = train_test(net, 1, criterion, optimizer, crv_iter) # always training
        crv_loss += [ losses[-1] ]
        print(" Test: ", e, '/', epochs)
        _, losses = train_test(net, 1, criterion, optimizer, tst_iter) # always training
        tst_loss += [ losses[-1] ] 
            
    return trn_loss, crv_loss, tst_loss

In [35]:
class Net1(nn.Module):
    def conv2d_output_shape(in_size, kernel, padding=1, stride=1):
        return ((in_size-kernel+2*padding)/stride) + 1
    
    def __init__(self):
        super(Net1, self).__init__()
        
        # 1 canal d'entrée, 2 canaux de sortie, convolution par un kernel 5x5
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2, padding=2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2, padding=2)
        
        
        lin_shape = int(np.floor(conv2d_output_shape(input_size, 5, padding=2, stride=2)))
        
        # les parametres du percetron multi-couche: y = Wx + b
        self.fc1 = nn.Linear(2*lin_shape*lin_shape, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling sur une fenêtre de 3x3 (par défaut stride=taille de la fenêtre)
        x = F.relu(self.conv1(x))
        
        print(10,x.shape)
        
        x = torch.flatten(x, start_dim=1) # transforme le tenseur en vecteur (dim 0 est celle du batch)
        # Perceptron
        x = F.relu(self.fc1(x))
        x = nn.functional.log_softmax(self.fc2(x)) # P(y|x)
        return x
    
# import data
batch_size = 32
input_size = 28
trn_loader, val_loader, tst_loader = load_data_fashion_mnist(batch_size, resize=input_size)
print('trn_loader: ', len(trn_loader))
print('val_loader: ', len(val_loader))
print('tst_loader: ', len(tst_loader))

# Net
net1 = Net1()
print(net1)
print(net1)


criterion = nn.NLLLoss()
optimizer = optim.SGD(net1.parameters(), lr=0.001)
epochs = 5

print("net1: trn_crval_tst")
trn_losses, val_losses, tst_losses = trn_crval_tst(net1, epochs, criterion, optimizer, trn_loader, val_loader, tst_loader)
plt.title("Loss evolution trn_crval_tst")
plt.plot(trn_losses, label='trn_loss')
plt.plot(val_losses, label='val_losses')
plt.plot(tst_losses, label='tst_losses')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

trn_loader:  1500
val_loader:  375
tst_loader:  313
0 14.5
1 14
Net1(
  (conv1): Conv2d(1, 2, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (fc1): Linear(in_features=392, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)
Net1(
  (conv1): Conv2d(1, 2, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (fc1): Linear(in_features=392, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)
net1: trn_crval_tst
 Train:  1 / 5
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])


<ipython-input-35-2f1e86b9a96d>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.log_softmax(self.fc2(x)) # P(y|x)


10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch.Size([32, 2, 14, 14])
10 torch

KeyboardInterrupt: 

In [32]:
np.floor(8.9)

8.0